In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoNBEATS
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import RNN

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
class FixedModelNBEATSProcessor:
    def __init__(self, dates=[]):
        self.dates = dates
        self.forecasts = []
        self.eval_pairs = []
        self.input_size = None
        self.nf = None
        self.config = None
        self.testset = None
        self.maes = []
        self.mses = []
        self.mapes = []
        self.nmses = []
        self.reference_dates = {}
        self.metrics_df = pd.DataFrame(columns=["Unique_id", "Reference Date", "MAE", "MSE", "MAPE", "NMSE"])
        self.metrics_display_df = pd.DataFrame()
        self.display_df = pd.DataFrame(columns=["Unique_id", "Reference Date", "Target End Date", "GT", "Quantile", "Prediction"])
        self.wis_df = pd.DataFrame()
        self.model = None
        self.selected_input_size = None
        self.unique_ids = []

    def create_fixed_model(self, df_train, df_test, test_ids, h, freq, level=[]):
        input_length = df_train.groupby("unique_id").size().min()
        max_input_size = max(8, input_length - h - 1)
        self.testset = df_test

        def config(trial):
            return {
                "input_size": trial.suggest_int("input_size", 8, max_input_size),
                "stack_types": ["trend", "seasonality"],
                "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
                "random_seed": trial.suggest_int("random_seed", 1, 99999),
                "accelerator": "gpu",
                "devices": 2,
                "strategy": "auto"
            }

        self.config = config

        if not level:
            nf = NeuralForecast(models=[AutoNBEATS(h=h, backend="optuna", config=self.config)], freq=freq)
        else:
            nf = NeuralForecast(models=[AutoNBEATS(h=h, backend="optuna", loss=MQLoss(level=level), config=self.config)], freq=freq)

        nf.fit(df=df_train)
        self.nf = nf
        self.model = nf.models[0].model
        self.selected_input_size = self.model.hparams['input_size']

    def sliding_batch_by_index(self, h: int = 4) -> pd.DataFrame:
        uid_dates = {
            uid: g["ds"].sort_values().to_numpy()
            for uid, g in self.testset.groupby("unique_id")
        }
        T = min(len(d) for d in uid_dates.values())
        s = self.selected_input_size
        results = []

        for t in tqdm(range(s, T - h + 1), desc="Batched by index t"):
            batch_records, uid_refs = [], []

            for uid, dates in uid_dates.items():
                ref_date   = dates[t - 1]            
                first_fcst = dates[t]                
                win_dates  = dates[t - s : t]       

                df_series  = self.testset[self.testset["unique_id"] == uid]
                df_input   = df_series[df_series["ds"].isin(win_dates)].copy()

                if len(df_input) == s:
                    df_input["unique_id"] = uid
                    batch_records.append(df_input)
                    uid_refs.append((uid, ref_date, first_fcst))

            if not batch_records:
                continue

            batch_df        = pd.concat(batch_records)
            batch_forecasts = self.nf.predict(df=batch_df).set_index("ds")

            # ---- split + evaluate per-uid ---------------------------------
            for uid, ref_date, first_fcst in uid_refs:
                fcast = batch_forecasts[batch_forecasts["unique_id"] == uid]
                series_df = self.testset[self.testset["unique_id"] == uid]
                gt        = series_df[series_df["ds"].isin(fcast.index)].set_index("ds")
        
                self.forecasts   = [fcast]
                self.eval_pairs  = [(fcast, gt)]
                self.dates       = [pd.to_datetime(ref_date).strftime("%Y-%m-%d")]
                self.unique_ids  = [uid]
                
                fcast = fcast.reset_index()
                gt = gt.reset_index()

                # Merge on both 'unique_id' and 'ds'
                merged_df = pd.merge(fcast, gt, on=['unique_id', 'ds'], how='inner')
                
                ref_date_str = pd.to_datetime(ref_date).strftime("%Y-%m-%d")  # optional formatting
                merged_df["Reference Date"] = ref_date_str
                
                results.append(merged_df)
                
        return pd.concat(results, ignore_index=True)


    def calculate_metrics(self):
        for forecast_df, truth_df in self.eval_pairs:
            y_true = truth_df.iloc[:, 1]
            y_pred = forecast_df.iloc[:, 0]
            self.maes.append(mean_absolute_error(y_true, y_pred))
            self.mses.append(mean_squared_error(y_true, y_pred))
            self.mapes.append(mean_absolute_percentage_error(y_true, y_pred))
            self.nmses.append(self.mses[-1] / np.var(y_true))

    def create_metrics_df(self):
        for i in range(len(self.dates)):
            self.metrics_df.loc[len(self.metrics_df)] = [
                self.unique_ids[i],
                self.dates[i],
                self.maes[i],
                self.mses[i],
                self.mapes[i],
                self.nmses[i]
            ]

    def create_display_df(self):
        records = []
        testset_indexed = self.testset.set_index(["unique_id", "ds"])

        for i, forecast_df in enumerate(tqdm(self.forecasts, desc="Building display_df")):
            uid = forecast_df["unique_id"].iloc[0]
            reference_date = self.dates[i]

            try:
                gt_series = testset_indexed.loc[uid]["y"]
            except KeyError:
                gt_series = pd.Series()

            for col in forecast_df.columns:
                if col == "unique_id":
                    continue

                if "lo" in col:
                    number = int(col.split("-")[-1])
                    alpha = 1 - (number / 100)
                    quantile = alpha / 2
                elif "hi" in col:
                    number = int(col.split("-")[-1])
                    alpha = 1 - (number / 100)
                    quantile = 1 - (alpha / 2)
                elif col in ["AutoNBEATS", "AutoNBEATS-median"]:
                    quantile = 0.5
                else:
                    continue

                df_col = forecast_df[[col]].copy()
                df_col.rename(columns={col: "Prediction"}, inplace=True)
                df_col["Unique_id"] = uid
                df_col["Reference Date"] = reference_date
                df_col["Target End Date"] = df_col.index
                df_col["GT"] = df_col.index.map(gt_series.get)
                df_col["Quantile"] = quantile
                records.append(df_col)

        self.display_df = pd.concat(records)[
            ["Unique_id", "Reference Date", "Target End Date", "GT", "Quantile", "Prediction"]
        ].sort_values(by=["Unique_id", "Reference Date", "Target End Date", "Quantile"]).reset_index(drop=True)

    def calculate_pointwise_metrics(self):
        records = []
        print("Calculating pointwise metrics...")

        for i in tqdm(range(len(self.eval_pairs)), desc="Computing pointwise errors"):
            forecast_df = self.forecasts[i]
            truth_df = self.eval_pairs[i][1]
            reference_date = self.dates[i]
            unique_id = self.unique_ids[i]

            pred_series = forecast_df["AutoNBEATS-median"]
            true_series = truth_df.iloc[:, 1]

            for target_date in pred_series.index:
                y_pred = pred_series.loc[target_date]
                y_true = true_series.loc[target_date]

                mae = abs(y_pred - y_true)
                mse = (y_pred - y_true) ** 2
                mape = abs((y_pred - y_true) / y_true) if y_true != 0 else np.nan
                nmse = mse / np.var(true_series) if np.var(true_series) != 0 else np.nan

                records.append({
                    "Unique_id": unique_id,
                    "Reference Date": reference_date,
                    "Target End Date": target_date,
                    "GT": y_true,
                    "MAE": mae,
                    "MSE": mse,
                    "MAPE": mape,
                    "NMSE": nmse
                })

        self.metrics_display_df = pd.DataFrame(records)

    def compute_wis(self):
        if self.display_df.empty:
            raise ValueError("display_df is empty. Run create_display_df() first.")

        df = self.display_df.sort_values(by=["Unique_id", "Reference Date", "Target End Date", "Quantile"])
        records = []
        grouped = df.groupby(["Unique_id", "Reference Date", "Target End Date"])

        print("Computing WIS for each forecasted point...")
        for (uid, ref_date, tgt_date), group in tqdm(grouped, desc="Computing WIS"):
            gt = group["GT"].iloc[0]
            preds = group.set_index("Quantile")["Prediction"]

            if 0.5 not in preds.index:
                continue

            ae = abs(preds[0.5] - gt)
            quantiles = sorted(q for q in preds.index if q != 0.5)
            n = len(quantiles) // 2
            interval_scores = []

            for i in range(n):
                lo_q = quantiles[i]
                hi_q = quantiles[-(i + 1)]
                lo = preds[lo_q]
                hi = preds[hi_q]
                alpha = hi_q - lo_q

                interval_score = (
                    (hi - lo)
                    + (2 / alpha) * max(lo - gt, 0)
                    + (2 / alpha) * max(gt - hi, 0)
                )
                interval_scores.append(interval_score)

            wis = (ae + np.sum(interval_scores)) / (1 + len(interval_scores))
            records.append({
                "Unique_id": uid,
                "Reference Date": ref_date,
                "Target End Date": tgt_date,
                "GT": gt,
                "WIS": wis
            })

        self.wis_df = pd.DataFrame(records)

In [6]:
test_ids = [f"Y_{i}" for i in test_indices]

In [7]:
processor = FixedModelNBEATSProcessor()
processor.create_fixed_model(
    df_train, df_test_all, test_ids, h=4, freq="W-SAT",
    level=[10, 20, 30, 40, 50, 60, 70, 80, 85, 90, 95]
)

[I 2025-06-24 18:00:45,694] A new study created in memory with name: no-name-7b4c90cf-d02f-4758-8102-16ad9937a094
[rank: 0] Seed set to 8584
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 23     | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [8]:
results = []

In [9]:
processor.selected_input_size

36

In [10]:
all_results = processor.sliding_batch_by_index(h=4)

Batched by index t:   0%|          | 0/21 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:   5%|▍         | 1/21 [00:33<11:08, 33.40s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:  10%|▉         | 2/21 [01:06<10:26, 32.96s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

Batched by index t:  90%|█████████ | 19/21 [10:28<01:06, 33.38s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:  95%|█████████▌| 20/21 [11:01<00:33, 33.42s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t: 100%|██████████| 21/21 [11:34<00:00, 33.07s/it]


In [11]:
all_results.to_csv('NBEATS_concatenated.csv')

In [12]:
all_results.iloc[0:12,:].to_csv('NBEATS_sample.csv')

In [27]:
sample = pd.read_csv('NBEATS_sample.csv')

In [30]:
sample = pd.read_csv('NBEATS_concatenated.csv')

In [31]:
df = sample.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoNBEATS-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 181440/181440 [01:28<00:00, 2058.71it/s]


In [32]:
pointwise_df

,Unique_id,Reference Date,Target End Date,GT,Prediction,MAE,MSE,MAPE,NMSE,WIS
0,Y_1,2011-05-21,2011-05-28,0.168067,0.088538,0.079529,0.006325,0.473196,NaN,0.412621
1,Y_1,2011-05-21,2011-06-04,0.065660,0.087941,0.022281,0.000496,0.339342,NaN,0.155825
2,Y_1,2011-05-21,2011-06-11,0.123609,0.088144,0.035465,0.001258,0.286911,NaN,0.224298
3,Y_1,2011-05-21,2011-06-18,0.051706,0.089422,0.037716,0.001422,0.729423,NaN,0.209427
4,Y_1,2011-05-28,2011-06-04,0.065660,0.176790,0.111130,0.012350,1.692505,NaN,0.525327
...,...,...,...,...,...,...,...,...,...,...
181435,Y_9999,1957-02-16,1957-03-16,0.000000,1.008817,1.008817,1.017712,NaN,NaN,5.105099
181436,Y_9999,1957-02-23,1957-03-02,0.000000,1.999395,1.999395,3.997582,NaN,NaN,10.320213
181437,Y_9999,1957-02-23,1957-03-09,0.000000,2.002795,2.002795,4.011188,NaN,NaN,10.310913
181438,Y_9999,1957-02-23,1957-03-16,0.000000,2.005672,2.005672,4.022720,NaN,NaN,10.289644


In [33]:
pointwise_df.to_csv('NBEATS_ALL_STATS.csv')

In [1]:
wis_dfs = [pointwise_df.iloc[i::4].reset_index(drop=True) for i in range(4)]

NameError: name 'pointwise_df' is not defined

In [ ]:
np.mean(wis_dfs[3]['WIS'].values)